In [1]:
import pandas as pd
from neo4j import GraphDatabase
from IPython.display import display

## Create Sub Data Set
### Filter by times range
- start_date = '2021-05-01' 
- end_date = '2021-09-30'

In [2]:
start_date = '2021-05-01'
end_date = '2021-09-30'

In [4]:
nasdaq100 = pd.read_csv("/user/projects/project-3-techChanakya/data/NASDAQ_100_Data_From_2010.csv",sep="\t")

In [5]:
sub_nasdaq100 = nasdaq100[(nasdaq100['Date'] >= start_date) & (nasdaq100['Date'] <= end_date)] 

print(sub_nasdaq100.shape)

(9384, 8)


In [6]:
sub_nasdaq100.to_csv('sub_nasdaq100.csv', index = False)

### Neo4j Setup

In [7]:
uri = "neo4j://neo4j:7473"
user = "neo4j"
password = "ucb_mids_w205"

driver = GraphDatabase.driver(uri=uri, auth=(user,password))

In [8]:
sub_nasdaq100.sort_values(by=['Name', 'Date'], inplace=True)

pivot_df = sub_nasdaq100.pivot(index='Date', columns='Name', values='Close')

# daily percent changes/returns in stocks
returns = pivot_df.pct_change().dropna()

# This is the Pearson correlation between stock returns
correlation_matrix = returns.corr()

/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [10]:
def create_correlation(tx, stock1, stock2, weight):
    tx.run("""
        MATCH (s1:Stock {name: $stock1}), (s2:Stock {name: $stock2})
        MERGE (s1)-[r:CORRELATED_WITH]->(s2)
        SET r.weight = $weight
    """, stock1=stock1, stock2=stock2, weight=weight)

with driver.session() as session:
    for stock1 in correlation_matrix.columns:
        for stock2 in correlation_matrix.columns:
            if stock1 != stock2:
                corr = correlation_matrix.loc[stock1, stock2]
                if corr >= 0.5:
                    session.write_transaction(create_correlation, stock1, stock2, float(corr))

/tmp/ipykernel_304/1371621696.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_correlation, stock1, stock2, float(corr))


In [ ]:
## Cypher Code
## MATCH (s1:Stock)-[r:CORRELATED_WITH]->(s2:Stock) RETURN s1, r, s2